# Pytorch Transfer Learning with Adam Optimizer

Neural Networks can take a long time to train. Additionally, techniques like [model pruning](https://towardsdatascience.com/pruning-deep-neural-network-56cae1ec5505) and other optimizations sometimes take many trials and errors due to a large number of hyperparameters. However, it can often be necessary to do those model optimizations to achieve both your performance and optimizing goals. Luckily, though, pruned (sparsified) Neural Networks in the computer vision and natural language space [transfer learn](https://towardsdatascience.com/a-comprehensive-hands-on-guide-to-transfer-learning-with-real-world-applications-in-deep-learning-212bf3b2f27a) very well. 

To make it easier to use pruned models, Neural Magic's ML team is actively creating pruned versions of popular models and datasets and updating the repo with them. Also, these models are tested thoroughly with the [Neural Magic Inference Engine](https://neuralmagic.com/) to ensure performance.

This notebook provides an easy step by step walkthrough for downloading a recalibrated model from the Neural Magic Model Repo and using it for transfer learning. Below we will go through the following steps:
1. Environment Setup
2. Model Selection
3. Model and Dataset Setup
4. Transfer Learning
5. Export to ONNX

Reading through this notebook will be reasonably quick to gain an intuition for what is happening. Rough time estimates for transfer learning are given, note since we are training with the Pytorch CPU implementation it will be much slower than a GPU:
- 15 minutes on a GPU
- 3 hours on a laptop CPU

## Environment Setup

Below we try to add the project folder to the PYTHONPATH environment variable for our execution. If this does not work, we will need to install neuralmagicML into the system using `pip install ./` when you are located at the root of the folder.

Additionally, please be sure to install from the requirements.txt file located at the root before running: `pip install -r ./requirements.txt`

In [ ]:
import sys
import os

notebook_name = "transfer_learning_adam_pytorch"

# environment setup for ease of use (puts neuralmagicML into the python package path)
if "WORKBOOK_DIR" not in globals():
    WORKBOOK_DIR = os.getcwd()

package_path = os.path.abspath(
    os.path.join(os.path.expanduser(WORKBOOK_DIR), os.pardir)
)
sys.path.extend([package_path])

print("added {} to PYTHONPATH".format(package_path))
print("working out of {}".format(WORKBOOK_DIR))

## Model Selection

There can be a lot of models available in repositories, so a simple UI is provided to make this selection process easier. Within the UI, filters can be applied for models trained in/on specific domains or datasets. Each network architecture listed out will also include options for the dataset it was trained on and the type. The type refers to how the models were trained and/or recalibrated. Specifically:
- base - baseline model, trained generally as in the original paper
- recal - a recalibrated model, it is recalibrated to the point of fully recovering the baseline model's metrics
- recal-perf - a recalibrated model, it is recalibrated for performance to the point of recovering 99% of the baseline model's metrics

In [ ]:
from neuralmagicML.nbutils import ModelSelectWidgetContainer

print("creating ui...")
container = ModelSelectWidgetContainer("pytorch")
display(container.create())

## Model and Dataset Setup

By default, we use the [Imagewoof](https://github.com/fastai/imagenette) dataset to transfer learn to (a dataset consisting of 10 classes of dogs). This dataset is used to show how to transfer learn on a simple dataset quickly. If you would like to try out transfer learning on your own dataset, then replace the appropriate lines with your own:
- `num_classes = 10`
- `class_type = "single"`
- `train_dataset = ImagewoofDataset(dataset_root, train=True)`
- `val_dataset = ImagewoofDataset(dataset_root, train=False)`

More information for creating and working with Pytorch datasets can be found [here](https://pytorch.org/tutorials/beginner/data_loading_tutorial.html). Take care to keep the variable names the same, as the rest of the notebook is set up according to those.

In [ ]:
from neuralmagicML.pytorch.datasets import ImagewoofDataset
from neuralmagicML.pytorch.models import ModelRegistry
from neuralmagicML.utils import clean_path

#######################################################
# Define the number of classes to transfer learn to below
#######################################################
num_classes = 10
class_type = "single"  # use single for softmax output, multi for sigmoid output
print(
    "transfer learning to {} classes and class_type {}".format(num_classes, class_type)
)

repo_model = container.selected_model
print("\nloading model {} ...".format(repo_model.root_path))
model = ModelRegistry.create(
    repo_model.registry_key,
    pretrained=repo_model.desc,
    num_classes=num_classes,
    class_type=class_type,
)
model_name = model.__class__.__name__
print(model)

#######################################################
# Define your train and validation datasets below
#######################################################
dataset_root = clean_path(os.path.join(".", notebook_name, "datasets"))

print("\nloading train dataset...")
train_dataset = ImagewoofDataset(dataset_root, train=True)
print(train_dataset)

print("\nloading val dataset...")
val_dataset = ImagewoofDataset(dataset_root, train=False)
print(val_dataset)

## Transfer Learning

Now that the model and datasets are chosen and set up, we will begin transfer learning from the given model onto the dataset. The library to enable this is designed to be easily plugged into nearly any training setup for Pytorch. Below we provide an example of how an integration looks. Note, only a handful of these lines are needed to be able to integrate fully.

1. Create a [`ConstantKSModifier`](); handles keeping the sparsity the same for any sparsified layers
2. Create a [`ScheduledModifierManager`](); used in combination with the `ConstantKSModifier` and the `ScheduledOptimizer`
3. Create a [`ScheduledOptimizer`](); handles updating the Pytorch objects that modify the training process. It wraps the original optimizer that was used to modify the training process/graph, and should be used in place of that. IE, optimizer.step() must be called on `ScheduledOptimizer` and not the original.
4. Call into the `ScheduledOptimizer` for `epoch_start()` and `epoch_end()` while training. These calls mark when an epoch has started and after training for an epoch has ended, respectively.

Once the training objects are created (optimizer, loss function, etc.), a `ConstantKSModifier`, `ScheduledModifierManager`, and `ScheduledOptimizer` are instantiated. Almost all logging and updates are done through `Tensorboard` for this notebook. The use of `Tensorboard` is optional, and other loggers (as well as not using a logger) are available. Finally, regular training and testing code is used to go through the process.

In [ ]:
import math
from tqdm import auto
import torch
from torch.utils.data import DataLoader
from torch.optim import Adam
from neuralmagicML.pytorch.utils import (
    CrossEntropyLossWrapper,
    TopKAccuracy,
    ModuleTrainer,
    ModuleTester,
    TensorboardLogger,
)
from neuralmagicML.utils import create_unique_dir, clean_path


#######################################################
# Necessary imports for running transfer learning
#######################################################
from neuralmagicML.pytorch.recal import (
    ScheduledModifierManager,
    ScheduledOptimizer,
    ConstantKSModifier,
)


# setup device, data loaders, loss, optimizer
device = "cuda" if torch.cuda.is_available() else "cpu"
batch_size = 128
train_data_loader = DataLoader(
    train_dataset, batch_size, shuffle=True, pin_memory=True, num_workers=8
)
val_data_loader = DataLoader(
    val_dataset, batch_size, shuffle=False, pin_memory=True, num_workers=8
)
loss = CrossEntropyLossWrapper(extras={"top1acc": TopKAccuracy(1)})
optim = Adam(model.parameters())
print("device:{} batch_size:{} loss:{}".format(device, batch_size, loss))

tensorboard_model_path = create_unique_dir(
    os.path.join(".", "tensorboard-logs", notebook_name, model_name)
)
loggers = [TensorboardLogger(tensorboard_model_path)]
print("logging at {}".format(tensorboard_model_path))


#######################################################
# First lines that must be substituted in training code
# We create a ConstantKSModifier to hold the sparsity constant on all layers
# Additionally we create a modifier manager as well as a scheduled optimizer
# These will allow us to transfer learn with sparsity
# The loggers can be left out if desired
#######################################################
manager = ScheduledModifierManager([ConstantKSModifier(layers="__ALL__")])
optim = ScheduledOptimizer(
    optim,
    model,
    manager,
    steps_per_epoch=math.ceil(len(train_dataset) / batch_size),
    loggers=loggers,
)
print("created modifier, manager, and optimizer")


# we use prewritten trainers and testers to make the code more concise
trainer = ModuleTrainer(model, device, loss, optim, loggers=loggers)
tester = ModuleTester(model, device, loss, loggers=loggers)
model = model.to(device)

# startup tensorboard
%load_ext tensorboard
%tensorboard --logdir ./tensorboard-logs

# run initial validation for comparison
tester.run_epoch(val_data_loader, epoch=-1, show_progress=False)


#######################################################
# Final lines that must be substituted in training code
# We tell the ScheduledOptimizer when each epoch has started and ended
#######################################################
num_epochs = 20
for epoch in auto.tqdm(range(num_epochs), desc="transfer learning"):
    optim.epoch_start()

    trainer.run_epoch(train_data_loader, epoch, show_progress=False)
    tester.run_epoch(val_data_loader, epoch, show_progress=False)

    optim.epoch_end()

# delete so all modifiers are cleaned up before exporting
del optim
print("training completed")

## Export to ONNX

Now that the model is fully recalibrated, we need to export it to an ONNX format. The ONNX format is what is used by the Neural Magic Inference Engine. For Pytorch, exporting to ONNX is natively supported. Below we use a convenience class to handle exporting, the [`ModuleExporter`](). Once the model has saved as an ONNX file, it is ready to be used for inference with Neural Magic.

In [ ]:
from neuralmagicML.utils import clean_path
from neuralmagicML.pytorch.utils import ModuleExporter

export_path = clean_path(os.path.join(".", notebook_name, model_name))
exporter = ModuleExporter(model, export_path)
for batch in val_data_loader:
    sample_input = batch[0]
    break
exporter.export_onnx(sample_input)
print("exported onnx to {}".format(export_path))